In [91]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [92]:
# data = pd.read_csv('training_dataset/raw_dataset.csv')
# data = pd.read_csv('training_dataset/raw_scores_pvallog.csv')
# data = pd.read_csv('training_dataset/ionocyte_raw_dataset.csv')
data = pd.read_csv('training_dataset/ionocyte_scores_pvallog.csv')

In [93]:
# Prepare the feature matrix and target vector
# X = data.drop(['NAME', 'disease_ontology_label', 'group'], axis=1)
data['disease_ontology_label'] = (data['disease_ontology_label'] == 'COVID-19').astype(int)
X = data.drop(['NAME', 'disease_ontology_label'], axis=1)
y = data['disease_ontology_label']

# Display the number of features before feature selection
print(f"Total number of features before selection: {X.shape[1]}")

Total number of features before selection: 2105


In [94]:
# Downsampling to balance classes
# g = data.groupby('disease_ontology_label')
# data_balanced = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))

# # Update X and y after downsampling
# X = data_balanced.drop(['NAME', 'disease_ontology_label'], axis=1)
# y = data_balanced['disease_ontology_label']
# X = X.dropna()
# y = y[X.index]

# Display new class sizes
# print("Class sizes after balancing:")
# print(y.value_counts())

In [95]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Impute missing values
# imputer = SimpleImputer(strategy='mean')
# X_train = imputer.fit_transform(X_train)
# X_test = imputer.transform(X_test)


# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# kbest = 60 for pval0 and kbest = 20 for plvallog ionocyte
# Feature selection using SelectKBest with f_classif
selector = SelectKBest(score_func=f_classif, k=200)  # Select top 100 features
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

# # Dimensionality reduction with PCA
# pca = PCA(n_components=0.8)  # Retain 95% of variance
# X_train = pca.fit_transform(X_train)
# X_test = pca.transform(X_test)

# Display the number of features after PCA
# print(f"Total number of features after PCA: {X_train.shape[1]}")

c:\Users\atif7\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [ 322  864 1038 1568 1765 1766 1767 1768 1772 1775 1776 1777 1778 1779
 1780 1781 1782 1785 1786 1788 1789 1790 1791 1792 1796 1798 1800 1801
 1804 1805 1806 1807 1810 1812 1813 1815 1816 1817 1818 1819 1822 1823
 1824 1826 1828 1832 1833 1835 1836 1837 1838 1841 1842 1843 1844 1849
 1851 1852 1853 1856 1857 1860 1861 1863 1868 1869 1870 1876 1877 1878
 1879 1880 1881 1882 1883 1884 1885 1888 1889 1890 1891 1892 1894 1895
 1896 1898 1901 1903 1905 1906 1907 1909 1910 1911 1912 1913 1914 1915
 1916 1918 1919 1920 1921 1922 1923 1924 1927 1929 1930 1931 1933 1934
 1937 1940 1942 1943 1944 1948 1951 1953 1956 1959 1962 1963 1966 1968
 1969 1972 1973 1978 1980 1982 1983 1984 1985 1986 1990 1991 1994 1996
 1997 1999 2000 2001 2003 2005 2006 2011 2012 2013 2014 2015 2017 2019
 2022 2023 2026 2028 2029 2030 2032 2034 2036 2039 2040 2043 2044 20

In [96]:
# # Applying t-SNE to reduce dimensionality for visualization
# tsne = TSNE(n_components=2, random_state=42)
# X_train_tsne = tsne.fit_transform(X_train)

# # Visualize the results of t-SNE
# plt.figure(figsize=(8, 6))
# plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], c=y_train, cmap='viridis', alpha=0.5)
# plt.colorbar()
# plt.title('t-SNE visualization of training data')
# plt.xlabel('t-SNE Feature 1')
# plt.ylabel('t-SNE Feature 2')
# plt.show()

In [97]:
# Train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, max_depth = 20)
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("--------------------------------------------------")
print("Random Forest Classifier Results")
print("--------------------------------------------------")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

print(classification_report(y_test, y_pred))

--------------------------------------------------
Random Forest Classifier Results
--------------------------------------------------
Accuracy: 0.7777777777777778
Precision: 0.7603264490339774
Recall: 0.7480952380952381
F1 Score: 0.7530844155844156
              precision    recall  f1-score   support

           0       0.71      0.64      0.67        42
           1       0.81      0.85      0.83        75

    accuracy                           0.78       117
   macro avg       0.76      0.75      0.75       117
weighted avg       0.77      0.78      0.78       117

